<h1>Proyecto Tecnologías de Búsqueda en la Web</h1>
<h3>Integrantes</h3>
<ul><li>Sebastián Aranda</li><li>Felipe Santander</li></ul>

<h2>Librerías</h2>

In [104]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary, bleicorpus
from gensim.models import ldamodel
from gensim.models import Phrases

from collections import defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import matplotlib.pyplot as plt
import numpy as np

from random import randint
from __future__ import division

import pyLDAvis
import re
import os
import json
import operator

<h2>Creación del Corpus</h2>

<h6>Rutas a Datasets</h6>

In [105]:
roastme_dataset_path = "Corpus_builder_and_pre/Dataset/"
positive_dataset_path = "Corpus_builder_and_pre/Positivos/"
#balance_dataset_path = "Corpus_builder_and_pre/Neutros&Positivos/"

<h6>Construcción del Corpus</h6>

In [127]:
puncts = ".,:;?!()[]{}~+-\"\'#$%&\/*^|"
digits = "0123456789"
#Create bad words list
bad_words = []
with open('badwords_v2.txt','r') as bad_words_file:
    for word in bad_words_file:
        word = word.split('-')[0].strip()
        word = word.replace('\n','').decode('unicode_escape').encode('ascii','ignore')
        if word != '':
            bad_words.append(word)

#Create the corpus
corpus = list()
for name in os.listdir(roastme_dataset_path):
    if (len(corpus)>300):
        break
    if name.endswith('.json'):
        with open(roastme_dataset_path+'/'+name) as f:
            op_json = json.loads(f.read())
            try:
                for child in op_json[1]['data']['children']:    
                    #Extract the comment
                    comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                    #Delete links
                    comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    for sym in puncts:
                        comment_text = comment_text.replace(sym," ")
                    for num in digits:
                        comment_text = comment_text.replace(num," ")
                    
                    tokens_comment = [word for word in comment_text.lower().split()]
                    
                    if (randint(1,10)<=8):
                        tokens_comment.append(bad_words[randint(0,len(bad_words)-1)])
                        tokens_comment.append(bad_words[randint(0,len(bad_words)-1)])
                        
                    corpus.append(tokens_comment)
                        
                    try:
                        for child in child['data']['replies']['data']['children']:
                            comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                            comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                            for sym in puncts:
                                comment_text = comment_text.replace(sym," ")
                            for num in digits:
                                comment_text = comment_text.replace(num," ")
                        
                            tokens_comment = [word for word in comment_text.lower().split()]
                            
                            if (randint(1,10)<=8):
                                tokens_comment.append(bad_words[randint(0,len(bad_words)-1)])
                                tokens_comment.append(bad_words[randint(0,len(bad_words)-1)])
                                
                            corpus.append(tokens_comment)
                                
                    except Exception as e:
                        #print(e)
                        pass
                    
            except Exception as e:
                #print(e)
                pass

for name in os.listdir(positive_dataset_path):
    if (len(corpus)>600):
        break
    if name.endswith('.json'):
        with open(positive_dataset_path+'/'+name) as f:
            op_json = json.loads(f.read())
            try:
                for child in op_json[1]['data']['children']:    
                    #Extract the comment
                    comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                    #Delete links
                    comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    for sym in puncts:
                        comment_text = comment_text.replace(sym," ")
                    for num in digits:
                        comment_text = comment_text.replace(num," ")
                    
                    tokens_comment = [word for word in comment_text.lower().split()]        
                    corpus.append(tokens_comment)
                        
                    try:
                        for child in child['data']['replies']['data']['children']:
                            comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                            comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                            for sym in puncts:
                                comment_text = comment_text.replace(sym," ")
                            for num in digits:
                                comment_text = comment_text.replace(num," ")
                        
                            tokens_comment = [word for word in comment_text.lower().split()]
                            corpus.append(tokens_comment)
                                
                    except Exception as e:
                        pass
                    
            except Exception as e:
                pass

print("Dataset Size")
print(len(corpus))
#TODO: guardar corpus de texto con pickle

Dataset Size
605


<h6>Procesamiento del Corpus</h6>

In [128]:
#Remove lone letters
corpus = [[word for word in doc if len(word)>1] for doc in corpus]

#Remove stopwords
corpus = [[word for word in doc if word not in stopwords.words('english')] for doc in corpus]

# Lemmatize all words in documents.
lemmatizer = WordNetLemmatizer()
corpus = [[lemmatizer.lemmatize(word) for word in doc] for doc in corpus]

#Get Trigrams
#bigram = Phrases(corpus)
#trigram = Phrases(bigram[corpus])
#for idx in range(len(corpus)):
#    for token in trigram[corpus[idx]]:
#            if '_' in token:
#                corpus[idx].append(token)

<h6>Transformación del Corpus a espacio de vectores</h6>

In [129]:
#Create dictionary
dictionary = corpora.Dictionary(corpus)
#dictionary.filter_extremes(no_below=len(dictionary)*0.001, no_above=0.75)

#Convert documents to vectors
corpus = [dictionary.doc2bow(doc) for doc in corpus]

#Save corpus and dictionary
corpora.BleiCorpus.serialize('tmp/cyberbullying_corpus.lda-c', corpus)
dictionary.save('tmp/cyberbullying_dictionary.dict')

<h1>Características para Clasificación</h1>
<ul>
<li>Porcentaje de pertenencia al tópico generado de LDA en Roastme Dataset</li>
<li>Densidad de Badwords</li>
<li>TODO: Polaridad</li>
</ul>

In [ ]:
#Load the dictionary and corpus
if (os.path.exists('tmp/cyberbullying_dictionary.dict') and os.path.exists('tmp/cyberbullying_corpus.lda-c')):
    print('Creating dictionary...')
    %time dictionary = corpora.Dictionary.load('tmp/cyberbullying_dictionary.dict')
    print('Creating corpus...')
    %time corpus = corpora.BleiCorpus('tmp/cyberbullying_corpus.lda-c')
else:
    print("Create the dictionary and corpus first")

<h3>LDA Features</h3>

In [126]:
#Initialize a model
#print('Creating Tfidf model...')
#tfidf = gensim.models.TfidfModel(corpus)
#corpus_tfidf = tfidf[corpus]
#lda_model_tfidf = ldamodel.LdaModel(corpus_tfidf, num_topics=2, id2word=dictionary)

%time lda_model = ldamodel.LdaModel(corpus, num_topics=2, id2word=dictionary)
lda_model.save('tmp/cyberbullying_ldaModel.lda')
lda_model.print_topics(2,100)
#doc1bow = dictionary.doc2bow(doc1)
#print(lda_model[doc1bow])

#lda_features = []
#for docbow in corpus:
#    lda_features.append(lda_model[docbow][0])
#print(lda_features)

CPU times: user 8.32 s, sys: 112 ms, total: 8.44 s
Wall time: 8.54 s


[(0,
  u'0.112*"okay" + 0.093*"dog" + 0.081*"perfection" + 0.081*"closer" + 0.068*"get" + 0.049*"help" + 0.046*"u" + 0.004*"like" + 0.003*"look" + 0.003*"look_like" + 0.002*"one" + 0.002*"make" + 0.002*"love" + 0.002*"face" + 0.001*"guy" + 0.001*"wholesome" + 0.001*"feel" + 0.001*"thing" + 0.001*"think" + 0.001*"people" + 0.001*"say" + 0.001*"friend" + 0.001*"would" + 0.001*"good" + 0.001*"really" + 0.001*"time" + 0.001*"hair" + 0.001*"going" + 0.001*"post" + 0.001*"go" + 0.001*"life" + 0.001*"way" + 0.001*"as" + 0.001*"meme" + 0.001*"right" + 0.001*"got" + 0.001*"see" + 0.001*"never" + 0.001*"even" + 0.001*"need" + 0.001*"day" + 0.001*"bad" + 0.001*"know" + 0.001*"much" + 0.001*"school" + 0.001*"fuck" + 0.001*"better" + 0.001*"dick" + 0.001*"happy" + 0.001*"place" + 0.001*"kid" + 0.001*"though" + 0.001*"also" + 0.001*"still" + 0.001*"maybe" + 0.001*"brother" + 0.001*"damn" + 0.001*"trying" + 0.001*"night" + 0.001*"take" + 0.001*"ever" + 0.001*"head" + 0.001*"first" + 0.001*"fucking" +

In [47]:
print(lda_model[dictionary.doc2bow("bitch kid".split())])

[(0, 1.0)]


<h3>Bad Words Features</h3>

In [89]:
#Create bad words list
bad_words = []
with open('badwords','r') as bad_words_file:
    for word in bad_words_file:
        word = word.replace('\n','').decode('unicode_escape').encode('ascii','ignore')
        if word != '':
            bad_words.append(word)

bad_words_ids = [word_id for word_id, word in dictionary.iteritems() if word in bad_words]

bad_words_features = []
for docbow in corpus:
        bad_words_features.append(sum([freq for word_id,freq in docbow if word_id in bad_words_ids]))
print(bad_words_features)

ValueError: too many values to unpack

<h1>Validación Cruzada</h1>

In [ ]:
from sklearn.cross_validation import train_test_split
X = zip(lda_features,bad_words_features)
y = [1 for doc in corpus] #Supuesto todos son agresivos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .7)

from sklearn.neighbors import KNeighborsClassifier
KNeighborsClassifier.fit(X_train, y_train)
predictions = KNeighborsClassifier.predict(X_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions))